In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.layers import  *
from keras.models import  *


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train  = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/val.csv')

In [3]:
embedding_list = list(train.columns)

In [4]:
embedding_list.remove('is_trade')


In [5]:
train_x = train[embedding_list]
test_x = test[embedding_list]

train_y = train['is_trade']
test_y = test['is_trade']

In [6]:
emb_size = 20

### PnnModel

In [7]:
class PnnModel():
    def __init__(self,embedding_list,emb_size,train_x,train_y,test_x,test_y):
        self.embedding_list = embedding_list
        self.emb_size = emb_size
        self.train_x = train_x
        self.train_y = train_y
        self.test_x  = test_x
        self.test_y = test_y
        self.mxlen_set = self.get_mxlen_set()
        
    def builtModel(self):
        
        emb_list =[]
        inp_list = []
        product_list = []
        
        for feat in self.embedding_list:
            inp_temp = Input(shape=[1],name=feat)
            emb_temp = Flatten()(Embedding(self.mxlen_set[feat],self.emb_size)(inp_temp))
            inp_list.append(inp_temp)
            emb_list.append(emb_temp)
            
        for i in range(0,len(emb_list)):
            for j in range(i+1,len(emb_list)):
                temp = dot([emb_list[i],emb_list[j]],axes=1)
                product_list.append(temp)
        inp = concatenate(emb_list+product_list)
        
        inp = Dense(64,activation='relu')(inp)
        outp = Dense(1,activation='sigmoid')(inp)
        
        model = Model(inputs=inp_list,outputs=outp)
        
        model.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
        
        return model
        
                    
    def get_mxlen_set(self):
        X = {}
        for ebd in self.embedding_list:
            X[ebd] = np.max([self.train_x[ebd].max(),self.test_x[ebd].max()])+1
        return X
            
    def get_kears_data(self,data):
        X = {}
        for ebd in self.embedding_list:
            X[ebd] = np.array(data[ebd])
        return X
    
    def train(self,batch_size,epochs):
        self.model = self.builtModel()
        X_train = self.get_kears_data(self.train_x)
        self.model.fit(X_train,self.train_y,batch_size=batch_size,epochs=epochs,verbose=10)
            
    def predict(self,batch_size):
        X_val = self.get_kears_data(self.test_x)
        pred = self.model.predict(X_val,batch_size=batch_size)[:,0]
        return pred
    